In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import sys

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
if not "../src" in sys.path:
    sys.path.append("../src")

In [4]:
INPUTDIR = Path("..")/"data"
print(list(INPUTDIR.iterdir()))
train_df = pd.read_csv(INPUTDIR / "train.csv")
test_df = pd.read_csv(INPUTDIR / "test.csv")

[PosixPath('../data/test.csv'), PosixPath('../data/train.csv')]


In [5]:
train_df.shape

(1875, 5)

In [13]:
from common import remove_cap, align, word_regex
import re
word_regex2 = re.compile("(\W+)")

In [14]:
ht, st = train_df["HUMAN_TRANSCRIPTION SYSTEM_TRANSCRIPTION".split()].values[3]
ht, st

('τῆς ἡδεῖας πλεονεξίας πολλοὺς εἰς τὴν τῶν ἀλλ',
 'της ἐδίας πλσον ἐξιας πολλους ἐις τὴν τῶν ἀλ')

In [15]:
# import unicodedata
# def _remove_cap(char):
#     return unicodedata.normalize("NFC", unicodedata.normalize("NFD", char)[:1])
# def remove_cap(s):
#     return "".join([_remove_cap(char) for char in s])

In [35]:
import unicodedata
from collections import defaultdict
symbols = defaultdict(int)
for ht, mt in train_df["HUMAN_TRANSCRIPTION SYSTEM_TRANSCRIPTION".split()].values:
    spaces = [w.strip() for w in word_regex2.split(ht) if word_regex.match(w)]
    spaces = [s for s in spaces if len(s) > 0]
    if len(spaces) < 1:
        continue
    for w in spaces:
        for ch in w:
            name = unicodedata.name(ch)
            if name.find("GREEK") >= 0:
                symbols[(ch, name)] += 1
    #break
symbols

defaultdict(int,
            {('΄', 'GREEK TONOS'): 2,
             ('᾽', 'GREEK KORONIS'): 388,
             (';', 'GREEK QUESTION MARK'): 17,
             ('·', 'GREEK ANO TELEIA'): 67,
             ('᾿', 'GREEK PSILI'): 14})

In [128]:
from collections import Counter
def letters_with(texts, symbol="PSILI"):
    for line in texts:
        for i, char in enumerate(line):
            char = char.lower()
            name = unicodedata.name(char)
            if name.find(symbol) >= 0:
                letters = unicodedata.normalize("NFD", char)[:1]
                letters = unicodedata.normalize("NFC", letters)
                yield letters

and_regex = re.compile("\s+AND\s+")
def letters_with2(texts, symbol="PSILI"):
    for line in texts:
        for i, char in enumerate(line):
            name = unicodedata.name(char)
            if name.find("WITH") >= 0:
                # letters = unicodedata.normalize("NFD", char)[1:]
                # letters = unicodedata.normalize("NFC", letters)
                k = name.find("WITH")
                name = name[k+5:]
                for accent in and_regex.split(name):
                    yield accent



print(Counter(list(letters_with2(train_df.HUMAN_TRANSCRIPTION.values))))
Counter(list(letters_with2(train_df.SYSTEM_TRANSCRIPTION.values)))

Counter({'TONOS': 3232, 'PSILI': 2250, 'PERISPOMENI': 1754, 'VARIA': 1668, 'DASIA': 843, 'OXIA': 712, 'YPOGEGRAMMENI': 352, 'DIALYTIKA': 13})


Counter({'PSILI': 1291,
         'PERISPOMENI': 895,
         'VARIA': 1034,
         'TONOS': 1931,
         'DASIA': 408,
         'OXIA': 133,
         'YPOGEGRAMMENI': 128})

In [101]:
for symbol in [
        "PSILI", "TONOS", 
        "VARIA", "PERISPOMENI", "DASIA", "OXIA", "YPOGEGRAMMENI",
        "DIALYTIKA"
        ]:
    print("Symbol", symbol)
    count_symb = Counter(list(
        letters_with(
            train_df.HUMAN_TRANSCRIPTION.values,
            symbol=symbol
        )))
    print(count_symb)
    print()
    count_symb = Counter(list(
        letters_with(
            train_df.SYSTEM_TRANSCRIPTION.values,
            symbol=symbol
        )))
    print(count_symb)
    print("---")

Symbol PSILI
Counter({'ε': 726, 'α': 575, 'υ': 364, 'ι': 347, 'η': 101, 'ω': 68, 'ο': 58, '᾿': 14, 'ρ': 11})

Counter({'α': 398, 'υ': 394, 'ε': 369, 'ι': 89, 'η': 31, 'ω': 10})
---
Symbol TONOS
Counter({'ι': 791, 'ο': 596, 'α': 567, 'ε': 555, 'υ': 310, 'η': 264, 'ω': 149, '΄': 2})

Counter({'ι': 622, 'ο': 488, 'η': 248, 'α': 227, 'ε': 165, 'υ': 128, 'ω': 53})
---
Symbol VARIA
Counter({'ι': 477, 'α': 307, 'η': 281, 'ο': 262, 'ε': 174, 'υ': 119, 'ω': 48})

Counter({'ι': 308, 'η': 229, 'ο': 228, 'ε': 125, 'υ': 56, 'α': 51, 'ω': 37})
---
Symbol PERISPOMENI
Counter({'ω': 518, 'υ': 437, 'ι': 423, 'η': 313, 'α': 63})

Counter({'ω': 408, 'υ': 199, 'ι': 149, 'η': 139})
---
Symbol DASIA
Counter({'ο': 202, 'υ': 159, 'η': 152, 'ω': 101, 'ι': 91, 'ε': 52, 'α': 50, 'ρ': 36})

Counter({'ο': 156, 'υ': 89, 'η': 63, 'ω': 38, 'ι': 23, 'ε': 21, 'α': 18})
---
Symbol OXIA
Counter({'ε': 186, 'α': 124, 'ι': 116, 'ο': 111, 'υ': 75, 'η': 64, 'ω': 36})

Counter({'ε': 37, 'ω': 35, 'ι': 28, 'ο': 27, 'η': 6})
---
S

In [127]:
ht_train = train_df.HUMAN_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values

mt_train = train_df.SYSTEM_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values

def print_wc(texts, l=1):
    words_counter = Counter([remove_cap(w.lower()) for line in texts for w in line])
    d = {w: words_counter[w] for w in words_counter if len(w) == l}
    return d

ht_words = (print_wc(ht_train, 2))
print("ht:", sorted(ht_words))
print()
mt_words = (print_wc(mt_train, 2))
print("mt:", sorted(mt_words))
# β ζ λ π ς
print()
print(sorted([(m, mt_words[m]) for m in mt_words if m not in ht_words]))

ht: ['αβ', 'αγ', 'αι', 'αλ', 'αμ', 'αν', 'αρ', 'ας', 'αυ', 'αφ', 'βα', 'βη', 'βο', 'βω', 'γα', 'γε', 'γη', 'γι', 'γυ', 'γω', 'δε', 'δη', 'δι', 'δυ', 'δω', 'εδ', 'εθ', 'ει', 'εκ', 'ελ', 'εν', 'εξ', 'επ', 'ερ', 'ες', 'ετ', 'ευ', 'εφ', 'ζη', 'ηδ', 'ηλ', 'ην', 'ης', 'θα', 'θε', 'θν', 'θς', 'θυ', 'θω', 'ιε', 'ιθ', 'ιν', 'ις', 'ιω', 'κα', 'κε', 'κη', 'κν', 'κο', 'κς', 'κυ', 'κω', 'λα', 'λε', 'λη', 'λω', 'μα', 'με', 'μη', 'μο', 'μω', 'να', 'νη', 'νι', 'νυ', 'νω', 'οδ', 'οι', 'ολ', 'ον', 'ορ', 'ος', 'οτ', 'ου', 'πα', 'πε', 'πι', 'πο', 'ρα', 'ρε', 'ρη', 'ρι', 'σα', 'σε', 'ση', 'σι', 'συ', 'σω', 'τα', 'τε', 'τη', 'τι', 'το', 'τω', 'υμ', 'υπ', 'φα', 'φο', 'φυ', 'χε', 'χν', 'χς', 'χυ', 'χω', 'ψε', 'ψυ', 'ωδ', 'ων', 'ως']

mt: ['αα', 'αδ', 'αε', 'αη', 'αι', 'ακ', 'αλ', 'αν', 'αο', 'αρ', 'ας', 'ατ', 'αυ', 'αω', 'βα', 'βε', 'βη', 'βο', 'γα', 'γε', 'γη', 'γι', 'γν', 'γρ', 'γω', 'δα', 'δε', 'δη', 'δι', 'δκ', 'δν', 'δρ', 'δς', 'δσ', 'δυ', 'δω', 'εα', 'εγ', 'εε', 'εη', 'ει', 'εκ', 'ελ', 'εμ', 'εν', 'εξ',

In [1685]:

a, b = word_regex.split(ht), word_regex.split(st)
a_, b_ = [remove_cap(aword) for aword in a], [remove_cap(bword) for bword in b]
print(a, b)
result = align(a_, b_)
result.do_backtracing(a, b)

['τῆς', 'ἡδεῖας', 'πλεονεξίας', 'πολλοὺς', 'εἰς', 'τὴν', 'τῶν', 'ἀλλ'] ['της', 'ἐδίας', 'πλσον', 'ἐξιας', 'πολλους', 'ἐις', 'τὴν', 'τῶν', 'ἀλ']


[('τῆς', 'της', True),
 (None, 'ἐδίας', False),
 (None, 'πλσον', False),
 (None, 'ἐξιας', False),
 ('ἡδεῖας', None, False),
 ('πλεονεξίας', None, False),
 ('πολλοὺς', 'πολλους', True),
 ('εἰς', 'ἐις', True),
 ('τὴν', 'τὴν', True),
 ('τῶν', 'τῶν', True),
 (None, 'ἀλ', False),
 ('ἀλλ', None, False)]

In [1686]:

# rx.split(ht)

In [1687]:
remove_cap(ht)

'της ηδειας πλεονεξιας πολλους εις την των αλλ'

In [1688]:
from tqdm.auto import tqdm

In [1689]:
all_results = []

for ht, st in tqdm(train_df["HUMAN_TRANSCRIPTION SYSTEM_TRANSCRIPTION".split()].values):
    a, b = word_regex.split(ht), word_regex.split(st)
    a_, b_ = [remove_cap(aword) for aword in a], [remove_cap(bword) for bword in b]
    result = align(a_, b_)
    bt_result = result.do_backtracing(a, b)
    all_results.append(bt_result)


  0%|          | 0/1875 [00:00<?, ?it/s]

In [1690]:
from collections import defaultdict
matches_marks = defaultdict(list)
for sample in tqdm(all_results):
    for ht_word, st_word, is_match in sample:
        if is_match and ht_word != st_word:
            matches_marks[ht_word].append(st_word)


  0%|          | 0/1875 [00:00<?, ?it/s]

In [1691]:
from collections import Counter
matches_marks_grouped = {k: Counter(v) for k, v in matches_marks.items()}

In [1692]:
replacement_rules = {
    stw: htw
    for htw, stcounter in matches_marks_grouped.items()
    for stw in stcounter
}

In [1693]:
import unicodedata
for x in unicodedata.normalize("NFD", replacement_rules["του"]):
    print(unicodedata.name(x))

GREEK SMALL LETTER TAU
GREEK SMALL LETTER OMICRON
GREEK SMALL LETTER UPSILON
COMBINING GREEK PERISPOMENI


In [1694]:

train_df.HUMAN_TRANSCRIPTION.values

array(['ἐγγινομένα πάθη μὴ σβεννύντες ἀλλὰ τῆ εκλύσει',
       'τοῦ βίου τοῦ καθ ΄ εαυτοὺς πολλὰ γίνεσθαι συγχωροῦν',
       'τες ἐμπυρίζουσι τὸν ἀμπελῶνα ἀλλὰ καὶ ὁ διὰ', ...,
       'τα ανινα υπεμεινεν οπου γε και τον στρον κατε',
       'δεξατο ινα ημας τους υπο αμαρτιων προδε',
       'δομενους της καταρας ελευθερωση και βοα'], dtype=object)

In [1695]:
def collect_mismatch(pairs):
    a, b = list(zip(*pairs))
    a = [x for x in a if x is not None]
    b = [x for x in b if x is not None]
    return a, b

mm_pairs = []
for line in tqdm(all_results):
    pair = []
    for a, b, is_match in line:
        if is_match:
            if len(pair) > 0:
                mm_pairs.append(collect_mismatch(pair))
                pair = []
            continue
        pair.append((a, b))
    if len(pair) > 0:
        mm_pairs.append(collect_mismatch(pair))



  0%|          | 0/1875 [00:00<?, ?it/s]

In [1696]:
a, b = mm_pairs[1]

In [1697]:
a, b

(['βίου'], ['β', 'ου'])

In [1698]:
a_, b_ = [remove_cap(aword) for aword in a], [remove_cap(bword) for bword in b]
print(a, b)
result = align(a_, b_)
result.do_backtracing(a_, b_)

['βίου'] ['β', 'ου']


[(None, 'β', False), (None, 'ου', False), ('βιου', None, False)]

In [1699]:
hstats = []
mstats = []

for ht, mt in mm_pairs:
    hstats.append(len(ht))
    mstats.append(len(mt))

In [1700]:
np.percentile(hstats, 95), np.percentile(mstats, 95)

(7.0, 6.0)

In [1701]:
example = train_df.SYSTEM_TRANSCRIPTION.values[0]

In [1702]:
import spacy
from spacy.lang.el.examples import sentences 

nlp = spacy.load("el_core_news_sm")
#doc = nlp(sentences[0])

In [1703]:
doc = nlp(example)

In [1704]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

ἐγγενομεναπαδημησμεννωτες ADJ amod
ἀλλατῆε NOUN obj
κλησει VERB ROOT


In [1705]:


from datastruct import align_spaces
from datastruct import word2tokens
    
word2tokens("hello")

[CharToken(char='h', bow=True, eow=False),
 CharToken(char='e', bow=False, eow=False),
 CharToken(char='l', bow=False, eow=False),
 CharToken(char='l', bow=False, eow=False),
 CharToken(char='o', bow=False, eow=True)]

In [1706]:
alengths = [len(x) for x in a]
print(alengths)
ajoined = "".join(a)
bjoined = "".join(b)
a_, b_ = [remove_cap(aword) for aword in ajoined], [remove_cap(bword) for bword in bjoined]
print(a, b)
result = align(a_, b_)
result.do_backtracing(a_, b_)

[4]
['βίου'] ['β', 'ου']


[('β', 'β', True), ('ι', None, False), ('ο', 'ο', True), ('υ', 'υ', True)]

In [1707]:
a_, b_ = [remove_cap(aword) for aword in a], [remove_cap(bword) for bword in b]

aa = [t for w in a_ for t in word2tokens(w)]
bb = [t for w in b_ for t in word2tokens(w)]


result = align(aa, bb)
result.do_backtracing(aa, bb)

[(CharToken(char='β', bow=True, eow=False),
  CharToken(char='β', bow=True, eow=True),
  True),
 (CharToken(char='ι', bow=False, eow=False), None, False),
 (CharToken(char='ο', bow=False, eow=False),
  CharToken(char='ο', bow=True, eow=False),
  True),
 (CharToken(char='υ', bow=False, eow=True),
  CharToken(char='υ', bow=False, eow=True),
  True)]

In [1708]:
# result[2][16, 15]

In [1709]:
#tokens2words(tokens):

#split_by_word(tokens1, tokens2)


In [1710]:
corrected_pairs = []
#do_break=False
for ht, mt in tqdm(mm_pairs):
    a_, b_ = [remove_cap(aword) for aword in ht], [remove_cap(bword) for bword in mt]

    aa = [t for w in a_ for t in word2tokens(w)]
    bb = [t for w in b_ for t in word2tokens(w)]
    # print(aa, bb)

    result = align(aa, bb)
    alignment = result.do_backtracing(aa, bb)
    if len(alignment) == 0:
        continue
    #t1_ = [x[0] for x in alignment]
    #t2_ = [x[1] for x in alignment]
    #m = [x[2] for x in alignment]
    t1_, t2_, m = list(zip(*alignment))
    for words in split_by_word(t1_, t2_):
        
        t1, t2 = list(zip(*words))
        corrected_pairs.append((tokens2words(t1), tokens2words(t2)))
        #do_break =True
    #if do_break:
    #    break


  0%|          | 0/3067 [00:00<?, ?it/s]

In [1711]:
len(alignment), len(t1_), len(t2_), len(m)
# split_by_word(t1_, t2_, verbose=True)
#alignment
tokens2words(t1)

['ελευθερωση', 'και', 'βοα']

In [1712]:
ht_len, mt_len = list(zip(*[(len(x), len(y)) for x, y in corrected_pairs]))

In [1713]:
np.max(ht_len), np.max(mt_len), np.median(ht_len), np.median(ht_len), np.percentile(ht_len, 95), np.percentile(ht_len, 95)

(9, 7, 1.0, 1.0, 4.0, 4.0)

In [1714]:
ht_sum, mt_sum = list(zip(*[(sum([len(x)for x in xs]), sum([len(y)for y in ys])) for xs, ys in corrected_pairs]))

In [1715]:
for func in [np.max, np.median, lambda x: np.percentile(x, 95)]:
    print(func(ht_sum), func(mt_sum))


47 43
7.0 7.0
18.0 17.0


In [1716]:
from datastruct import BasicVocab
    
vocab = BasicVocab()
for line in tqdm(train_df.HUMAN_TRANSCRIPTION.values):
    words = word_regex.split(line)
    words_ = [remove_cap(word) for word in words]
    vocab.add_sentence(words)
    break

  0%|          | 0/1875 [00:00<?, ?it/s]

In [1717]:
size = 2
np.arange(len(words))

array([0, 1, 2, 3, 4, 5, 6])

In [1718]:
from common import sliding_window

list(sliding_window(words, 1)), words

([('ἐγγινομένα',),
  ('πάθη',),
  ('μὴ',),
  ('σβεννύντες',),
  ('ἀλλὰ',),
  ('τῆ',),
  ('εκλύσει',)],
 ['ἐγγινομένα', 'πάθη', 'μὴ', 'σβεννύντες', 'ἀλλὰ', 'τῆ', 'εκλύσει'])

In [1719]:
for n in range(1, 4):
    for word_seq in sliding_window(words, n):
        print(word_seq)
        break
    break

('ἐγγινομένα',)


In [1720]:
word_seq

('ἐγγινομένα',)

In [1721]:
from datastruct import AdvancedVocab
vocabs = {
    n: AdvancedVocab()
    for n in np.arange(4)+1
}
print(vocabs)
#
for line in tqdm(train_df.HUMAN_TRANSCRIPTION.values):
    words = word_regex.split(line)
    words = [w for w in words if len(w) > 0]
    words_ = [remove_cap(word) for word in words]
    # vocab.add_sentence(words)
    for n in range(1, 5):
        # print(type(n))
        for word_seq in sliding_window(words, n):

            # print(word_seq)
            vocabs[n].append(word_seq)
        # pass

    # break

{1: <datastruct.AdvancedVocab object at 0x7faa2f1f2e10>, 2: <datastruct.AdvancedVocab object at 0x7faa2f1f20d0>, 3: <datastruct.AdvancedVocab object at 0x7faa2f1f29d0>, 4: <datastruct.AdvancedVocab object at 0x7faa2f1f28d0>}


  0%|          | 0/1875 [00:00<?, ?it/s]

In [1722]:
for n in vocabs:
    vocabs[n].precompute()

In [1723]:
vocabs[2]

In [1724]:
# vocabs[2].find_nearby(self, query_str, max_rad=1, verbose=False)

In [1725]:
from datastruct import Word
distances = []
for ht, mt in tqdm(corrected_pairs):
    ht_word = Word(ht)
    mt_word = Word(mt)
    a, b = vocabs[2].bow2vector(ht_word.bow), vocabs[2].bow2vector(mt_word.bow)
    dist = np.abs(a - b).sum()
    d2 = np.sum([max(x, y) for x, y in zip(a, b)])
    distances.append(dist/d2)
    # break

  0%|          | 0/5511 [00:00<?, ?it/s]

In [1726]:
corrected_pairs[-10:]

[(['γενους', 'γεγονε', 'ανος'], ['γενθυγεγονενανος']),
 (['και', 'τι', 'λε'], ['οτιμε']),
 (['ημετεραν'], ['η', 'μετεραν']),
 (['υπεδυ', 'σαρκα'], ['υπεθσαρκα']),
 (['και', 'τ', 'αλλα', 'παν'], ['ειταλλαπαν']),
 (['υπεμεινεν'], ['υπεμειμεν']),
 (['οπου', 'γε', 'και'], ['οπουγεκαι']),
 (['τον', 'στρον'], ['ετριν']),
 (['υπο', 'αμαρτιων'], ['υποαμαρτιων']),
 (['ελευθερωση', 'και', 'βοα'], ['ελευθερωσηψβοα'])]

In [1727]:
# np.sum([i!= j for i, j in zip(ht_word.no_caps, mt_word.no_caps)])

In [1728]:
np.mean(distances), np.median(distances), np.percentile(distances, 95) # consider 95 percentile to be the default

(0.3623889085392239, 0.3333333333333333, 1.0)

In [1729]:
# mt_word.tokens_no_caps, word.tokens_no_caps

In [1730]:
# word.tokens_no_caps, word.no_caps

In [1731]:
print("source:", mt_word.sequence, "target:", ht_word.sequence)
for word, dist in vocabs[3].find_nearby(mt_word.sequence, max_rad=10, verbose=False):
    print(mt_word.sequence, word.sequence, dist)
    result = mt_word.distance_to(word)
    # result = align(mt_word.no_caps, word.no_caps)
    print(result.distance)
    alignment = result.do_backtracing("".join(mt_word.sequence), word.tokens)
    mt, rt, m = list(zip(*alignment))
    # print("aligned:", ["".join(word) for word in ])
    print("aligned:")
    for word, ref in align_spaces(mt, rt):
        print("".join(word), "<-", "".join([x.char for x in ref]))
    #for word in split_by_word(t1, t2):
    #    print(word)
    # break

source: ['ελευθερωσηψβοα'] target: ['ελευθερωση', 'και', 'βοα']
['ελευθερωσηψβοα'] ('ἔσθ', 'ἃ', 'βούλῃ') 5.0
12
aligned:
ελ <- 
ευθ <- ἔσθ
ερω <- 
σ <- ἃ
ηψβοα <- βούλῃ
['ελευθερωσηψβοα'] ('ελευθερωση', 'και', 'βοα') 4.0
3
aligned:
ελευθερωση <- ελευθερωση
ψ <- και
βοα <- βοα


In [1732]:
# result.show_backtrace()

In [1733]:
mt_word.sequence

['ελευθερωσηψβοα']

In [1734]:
a = vocabs[2].bow2vector(mt_word.bow)
b = vocabs[2].bow2vector(ht_word.bow)

In [1735]:
np.abs(a-b).sum()


4

In [1736]:
vocabs[3].find(['ελευθερωση', 'και', 'βοα'], verbose=True)

input seq: ['ελευθερωση', 'και', 'βοα'], ελευθερωσηκαιβοα
Checking index: 8808
word_index= 8837


[8837]

In [1737]:
print(np.sum([len(x) for x in ht_word.sequence]))
np.sum([v for k, v in ht_word.bow.items()])

16


16

In [1738]:
# vocabs[3].words[9321].sequence

In [1739]:
results = []
seqs = []
for k in vocabs[3].nocaps2index:
    result = align("ελευθερωσηκαιβο", k)
    results.append(result)
    seqs.append(k)

    

In [1740]:
# results[0].distance, results[1].distance, results[9073].distance

In [1741]:
#k = 9073
#results[k].do_backtracing("ελευθερωσηκαιβο", seqs[k])

In [1742]:
np.argmax([r.distance for r in results])

8808

In [1743]:
# from space_fixer import 
    

In [1744]:
for ht_line, mt_line in tqdm(train_df[["HUMAN_TRANSCRIPTION", "SYSTEM_TRANSCRIPTION"]].values[7:]):
    ht_words = word_regex.split(ht_line)
    mt_words = word_regex.split(mt_line)
    #words_ = [remove_cap(word) for word in words]
    #vocab.add_sentence(words)
    break
ht_words, mt_words

  0%|          | 0/1868 [00:00<?, ?it/s]

(['δικον', 'κτῆσιν', 'και', 'ἀπὸ', 'ἀρπαγῆς', 'καὶ', 'βίας', 'συλλεγόντες'],
 ['ικον', 'κτῆσιν', 'αποαρπαγης', 'καιβωας', 'συγεγοντεσ'])

In [1746]:
from space_fixer import build_path_matrix

dmatrix = build_path_matrix(mt_words, vocabs)

In [1747]:
# dist, closest_data = get_closest_data(mt_word, vocabs)

In [1748]:
# len(dmatrix[-1][1])

In [1750]:
#def convert_to_line_split(dmatrix):
from space_fixer import extract_paths


finished_paths = extract_paths(dmatrix)


In [1751]:
for i, path in enumerate(finished_paths):
    print(f"Path {i}")
    for (frag_dist, b, e, segs) in path:
        print(f"{b} -> {e}, dist={frag_dist}, {len(segs)} variants", mt_words[b:e])

Path 0
0 -> 2, dist=1, 1 variants ['ικον', 'κτῆσιν']
2 -> 4, dist=1, 1 variants ['αποαρπαγης', 'καιβωας']
4 -> 5, dist=4, 1 variants ['συγεγοντεσ']
Path 1
0 -> 2, dist=1, 1 variants ['ικον', 'κτῆσιν']
2 -> 3, dist=0, 1 variants ['αποαρπαγης']
3 -> 4, dist=1, 1 variants ['καιβωας']
4 -> 5, dist=4, 1 variants ['συγεγοντεσ']
Path 2
0 -> 1, dist=1, 5 variants ['ικον']
1 -> 2, dist=0, 1 variants ['κτῆσιν']
2 -> 4, dist=1, 1 variants ['αποαρπαγης', 'καιβωας']
4 -> 5, dist=4, 1 variants ['συγεγοντεσ']
Path 3
0 -> 1, dist=1, 5 variants ['ικον']
1 -> 2, dist=0, 1 variants ['κτῆσιν']
2 -> 3, dist=0, 1 variants ['αποαρπαγης']
3 -> 4, dist=1, 1 variants ['καιβωας']
4 -> 5, dist=4, 1 variants ['συγεγοντεσ']


In [1753]:
# alignment = result.do_backtracing("".join(mt_word.sequence), word.tokens)
# mt, rt, m = list(zip(*alignment))
# # print("aligned:", ["".join(word) for word in ])
# print("aligned:")
# for word, ref in align_spaces(mt, rt):
#     print("".join(word), "<-", "".join([x.char for x in ref]))

from space_fixer import resplit_paths

for k in resplit_paths(finished_paths, mt_words):
    variant = " ".join(k)



In [1754]:
# vocabs[1].index2seq[21]
ht_words

['δικον', 'κτῆσιν', 'και', 'ἀπὸ', 'ἀρπαγῆς', 'καὶ', 'βίας', 'συλλεγόντες']

In [1755]:
#for r, c, w in dmatrix[9][1][0][1]:  #dmatrix[1][1][0][1]:
#    print(w)
mt_words

['ικον', 'κτῆσιν', 'αποαρπαγης', 'καιβωας', 'συγεγοντεσ']

In [1757]:
for ht_line, mt_line in tqdm(train_df[["HUMAN_TRANSCRIPTION", "SYSTEM_TRANSCRIPTION"]].values[7:]):
    ht_words = word_regex.split(ht_line)
    mt_words = word_regex.split(mt_line)
    #words_ = [remove_cap(word) for word in words]
    #vocab.add_sentence(words)
    dmatrix = build_path_matrix(mt_words, vocabs)
    finished_paths = extract_paths(dmatrix)
    for k in resplit_paths(finished_paths, mt_words):
        variant = " ".join(k)
        print(variant)
    break
ht_words, mt_words

  0%|          | 0/1868 [00:00<?, ?it/s]

ικον κτῆσιν απο αρπαγης και βωας συ γεγοντε σ
ι κον κτῆσιν απο αρπαγης και βωας συ γεγοντε σ


(['δικον', 'κτῆσιν', 'και', 'ἀπὸ', 'ἀρπαγῆς', 'καὶ', 'βίας', 'συλλεγόντες'],
 ['ικον', 'κτῆσιν', 'αποαρπαγης', 'καιβωας', 'συγεγοντεσ'])